# 2D Channel With time-dependent boundary conditions
#### This example demonstrates the simulation of a flow in a 2D channel using constant and time dependent boundary conditions. 
[Link to Tutorials on Thetis Website](https://thetisproject.org/documentation.html)

We begin by importing Thetis and creating a rectangular mesh using the builtin Firedrake mesh utility. 
Our domain is 40 km long and 2 km wide, and we will generate 25 elements in the along-channel direction and 2 in the cross-channel direction:

In [ ]:
%matplotlib inline
from thetis import *

lx = 40e3
ly = 2e3
nx = 25
ny = 2
mesh2d = RectangleMesh(nx, ny, lx, ly)

We could use the built in [plot](https://firedrakeproject.org/firedrake.html#firedrake.plot.plot) function of firedrake to visualise the mesh.


In [ ]:
plot(mesh2d)

Next we define a bathymetry function in the 2D mesh using continuous linear elements, and set the bathymetry to a constant 20 m depth:

In [ ]:
P1_2d = FunctionSpace(mesh2d, 'CG', 1)
bathymetry_2d = Function(P1_2d, name='Bathymetry')
depth = 20.0
bathymetry_2d.assign(depth)

*Refer to the [Firedrake manual - Defining variational problems](https://firedrakeproject.org/variational-problems.html) for more information on mesh generation, functions and function spaces.*

We are now ready to create a 2D solver object, and set some options.

In [ ]:
solver_obj = solver2d.FlowSolver2d(mesh2d, bathymetry_2d)
options = solver_obj.options

First we set the options for the total duration and export intervals:

In [ ]:
t_end = 2 * 3600
t_export = 100.0
options.simulation_export_time = t_export
options.simulation_end_time = t_end

Next we define the time integrator, and set the time step, which can be chosed freely since Crank-Nicolson is unconditionally stable.

In [ ]:
options.timestepper_type = 'CrankNicolson'
options.timestep = 50.0

We will force the model with a constant volume flux at the right boundary (x=40 km) and impose a tidal volume flux on the left boundary (x=0 km).

Boundary condtitions are defined for each external boundary using their ID. In this example we will use the built in Firedrake mesh utility which assigns IDs 1, 2, 3, and 4 for the four sides of the rectangle.

In [1]:
left_bnd_id = 1
right_bnd_id = 2

At each boundary we need to define the external value of the prognostic variables, i.e. in this case the water elevation and velocity.The value should be either a Firedrake [Constant](https://firedrakeproject.org/firedrake.html#firedrake.constant.Constant) or a Firedrake [Function](https://firedrakeproject.org/firedrake.html#firedrake.function.Function) (in case the boundary condition is not uniform in space).

We store the boundary conditions in a dictionary.


In [ ]:
swe_bnd = {}
in_flux = 1e3
swe_bnd[right_bnd_id] = {'elev': Constant(0.0),
                         'flux': Constant(-in_flux)}

Above we set the water elevation to zero and prescribe a constant volume flux. The volume flux is defined as outward normal flux, i.e. a negative value stands for flow into the domain. Alternatively we could also prescribe the normal velocity (with key 'un') or the 2D velocity vector ('uv'). For all supported boundary conditions, see module [shallowwater_eq](https://thetisproject.org/thetis.html#module-thetis.shallowwater_eq).

In order to set time-dependent boundary conditions we first define a python function that evaluates the time dependent variable:

In [ ]:
def timedep_flux(simulation_time):
    """Time-dependent flux function"""
    tide_amp = -2e3
    tide_t = 12 * 3600.
    flux = tide_amp*sin(2 * pi * simulation_time / tide_t) + in_flux
    return flux

We then create a Constant object with the initial value, and assign it to the left boundary:

In [ ]:
tide_flux_const = Constant(timedep_flux(0))
swe_bnd[left_bnd_id] = {'flux': tide_flux_const}

Boundary conditions are now complete, and we assign them to the solver object:

In [ ]:
solver_obj.bnd_functions['shallow_water'] = swe_bnd

Note that if boundary conditions are not assigned for some boundaries (the lateral boundaries 3 and 4 in this case), Thetis assumes impermeable land conditions.

The only missing piece is to add a mechanism that re-evaluates the boundary condition as the simulation progresses. For this purpose we use the optional update_forcings argument of the [iterate()](https://thetisproject.org/thetis.html#thetis.solver2d.FlowSolver2d.iterate) method. update_forcings is a python function that updates all time dependent [Constants](https://firedrakeproject.org/firedrake.html#firedrake.constant.Constant) or [Functions](https://firedrakeproject.org/firedrake.html#firedrake.function.Function) used to force the model. In this case we only need to update tide_flux_const:
 
To visualise the results of the simulation we also plot the Elevation at selected time steps.

In [ ]:
def update_forcings(t_new):
    uv, elev = solver_obj.fields.solution_2d.split()
    """Callback function that updates all time dependent forcing fields"""
    tide_flux_const.assign(timedep_flux(t_new))
    plot_time = [1000,2000,4000,7000]
    for i in plot_time:
        if t_new == i:
            plot(elev)

Finally pass this callback to the time iterator:

In [ ]:
solver_obj.iterate(update_forcings=update_forcings)